In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt

import gc
import pickle as pickle


from sklearn.model_selection import KFold
from itertools import product
import tensorflow as tf

In [3]:
gc.collect()

items           = pd.read_csv('items.csv',usecols=["item_id", "item_category_id"])
item_categories = pd.read_csv('item_categories.csv')
shops           = pd.read_csv('shops.csv')
sales_train     = pd.read_csv('sales_train.csv.gz')
test            = pd.read_csv('test.csv.gz')

In [4]:
sales_train[['day','month', 'year']] = sales_train['date'].str.split('.', expand=True).astype(int)
#sales_train = sales_train[sales_train['year'].isin([2013,2014]) == False]
sales_train = sales_train[sales_train['date_block_num'] > 23]
sales_train = sales_train.set_index('item_id').join(items.set_index('item_id'))
sales_train.reset_index(inplace=True)

In [5]:
max_sales=1000
sums = sales_train.groupby('item_id')['item_cnt_day'].sum().reset_index().rename(columns={"item_cnt_day":"item_total_sales"}).sort_values(by='item_total_sales')

ids_keep = sums[(sums['item_total_sales'] > 0) & (sums['item_total_sales'] < max_sales)]['item_id'].unique()
#ids_keep = sums[(sums['item_total_sales'] > 0)]['item_id'].unique()

In [6]:
train_item_ids = sales_train['item_id'].unique()
#train_item_ids = np.setdiff1d(train_item_ids, ids_reject)
#train_item_ids = ids_keep
train_shop_ids = sales_train['shop_id'].unique()
test_item_ids = test['item_id'].unique()
test_shop_ids = test['shop_id'].unique()
train_blocks = sales_train['date_block_num'].unique()

#all_item_ids = np.unique(np.append(test_item_ids,train_item_ids))
all_item_ids = test_item_ids

#all_shop_ids = np.unique(np.append(train_shop_ids,test_shop_ids))
all_shop_ids = test_shop_ids



In [7]:
combinations = []

for dbn in range(np.min(train_blocks), np.max(train_blocks)+1):
    sales = sales_train[sales_train.date_block_num==dbn]
    #item_ids = np.intersect1d(sales.item_id.unique(), test_item_ids)
    item_ids = all_item_ids
    #dbn_combos = list(product(sales.shop_id.unique(), item_ids, [dbn]))
    dbn_combos = list(product(all_shop_ids, item_ids, [dbn]))
    for combo in dbn_combos:
        combinations.append(combo)
        
all_combos = pd.DataFrame(np.unique(np.vstack([combinations]), axis=0), columns=['shop_id','item_id','date_block_num'])

In [8]:
ys = sales_train.groupby(['shop_id', 'item_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_item_cnt_block"})

training = all_combos.merge(ys, on=['shop_id', 'item_id', 'date_block_num'], how='left').fillna(0)


training['shop_item_cnt_block'] = training['shop_item_cnt_block'].clip(0,20).astype('int8')

training = training.set_index('item_id').join(items.set_index('item_id'))
training.reset_index(inplace=True)

for col in ['item_id', 'shop_id', 'item_category_id']:
    training[col] = pd.to_numeric(training[col], downcast='unsigned')

In [9]:
dates = sales_train[['date_block_num', 'month', 'year']].drop_duplicates(['date_block_num', 'month', 'year'])

dates_dict = {}

for index,row in dates.iterrows():
    dates_dict[row['date_block_num']] = {"month": row['month'], "year": row['year']}
    
training['month'] = pd.to_numeric(training['date_block_num'].apply(lambda block: dates_dict[block]['month']), downcast='unsigned')
training['year'] = pd.to_numeric(training['date_block_num'].apply(lambda block: dates_dict[block]['year']), downcast='unsigned')



In [10]:
ys = sales_train.groupby(['item_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"item_cnt_block"})

training = training.merge(ys, on=['item_id', 'date_block_num'], how='left').fillna(0)

ys = sales_train.groupby(['shop_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_cnt_block"})

training = training.merge(ys, on=['shop_id', 'date_block_num'], how='left').fillna(0)


ys = sales_train.groupby(['item_category_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"category_cnt_block"})


training = training.merge(ys, on=['item_category_id', 'date_block_num'], how='left').fillna(0)


ys = sales_train.groupby(['shop_id', 'item_category_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_category_cnt_block"})

training = training.merge(ys, on=['shop_id', 'item_category_id', 'date_block_num'], how='left').fillna(0)


In [ ]:
training['item_cnt_block_mean'] = training.groupby(['date_block_num'])['item_cnt_block'].transform(np.mean)
training['item_cnt_block_min'] = training.groupby(['date_block_num'])['item_cnt_block'].transform(np.min)
training['item_cnt_block_max'] = training.groupby(['date_block_num'])['item_cnt_block'].transform(np.max)
training['item_cnt_block_std'] = training.groupby(['date_block_num'])['item_cnt_block'].transform(np.std)
training['item_cnt_block_med'] = training.groupby(['date_block_num'])['item_cnt_block'].transform(np.median)

In [ ]:
training['shop_cnt_block_mean'] = training.groupby(['date_block_num'])['shop_cnt_block'].transform(np.mean)
training['shop_cnt_block_min'] = training.groupby(['date_block_num'])['shop_cnt_block'].transform(np.min)
training['shop_cnt_block_max'] = training.groupby(['date_block_num'])['shop_cnt_block'].transform(np.max)
training['shop_cnt_block_std'] = training.groupby(['date_block_num'])['shop_cnt_block'].transform(np.std)
training['shop_cnt_block_med'] = training.groupby(['date_block_num'])['shop_cnt_block'].transform(np.median)

In [ ]:
training['category_cnt_block_mean'] = training.groupby(['date_block_num'])['category_cnt_block'].transform(np.mean)
training['category_cnt_block_min'] = training.groupby(['date_block_num'])['category_cnt_block'].transform(np.min)
training['category_cnt_block_max'] = training.groupby(['date_block_num'])['category_cnt_block'].transform(np.max)
training['category_cnt_block_std'] = training.groupby(['date_block_num'])['category_cnt_block'].transform(np.std)
training['category_cnt_block_med'] = training.groupby(['date_block_num'])['category_cnt_block'].transform(np.median)

In [ ]:
training['shop_category_cnt_block_mean'] = training.groupby(['date_block_num'])['shop_category_cnt_block'].transform(np.mean)
training['shop_category_cnt_block_min'] = training.groupby(['date_block_num'])['shop_category_cnt_block'].transform(np.min)
training['shop_category_cnt_block_max'] = training.groupby(['date_block_num'])['shop_category_cnt_block'].transform(np.max)
training['shop_category_cnt_block_std'] = training.groupby(['date_block_num'])['shop_category_cnt_block'].transform(np.std)
training['shop_category_cnt_block_med'] = training.groupby(['date_block_num'])['shop_category_cnt_block'].transform(np.median)

In [ ]:
training['shop_item_cnt_block_mean'] = training.groupby(['date_block_num'])['shop_item_cnt_block'].transform(np.mean)
training['shop_item_cnt_block_min'] = training.groupby(['date_block_num'])['shop_item_cnt_block'].transform(np.min)
training['shop_item_cnt_block_max'] = training.groupby(['date_block_num'])['shop_item_cnt_block'].transform(np.max)
training['shop_item_cnt_block_std'] = training.groupby(['date_block_num'])['shop_item_cnt_block'].transform(np.std)
training['shop_item_cnt_block_med'] = training.groupby(['date_block_num'])['shop_item_cnt_block'].transform(np.median)

In [ ]:
#https://maxhalford.github.io/blog/target-encoding-done-the-right-way/
#https://www.kaggle.com/vprokopev/mean-likelihood-encodings-a-comprehensive-study

from sklearn.model_selection import StratifiedKFold
columns = ["item_id", "shop_id", "item_category_id"]



y_train = training["shop_item_cnt_block"].values
folds = KFold(n_splits = 5, shuffle=True).split(training)

i=1
for in_fold_index, out_of_fold_index in folds:
    print("fold", i)
    #print(np.intersect1d(training.loc[in_fold_index]["shop_id"].unique(), training.loc[out_of_fold_index]["shop_id"].unique()))
    #print(len(in_fold_index))
    for column in columns:
        means = training.iloc[in_fold_index].groupby(column)['shop_item_cnt_block'].mean()
            #x_validation[column + "_mean_target"] = means\
        name = column + '_mean_encoding'
        training.loc[out_of_fold_index,name] = training.loc[out_of_fold_index][column].map(means)
    i+=1

In [11]:
training.fillna(0,inplace=True)

In [12]:

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
training.sample(10)

,item_id,shop_id,date_block_num,shop_item_cnt_block,item_category_id,month,year,item_cnt_block,shop_cnt_block,category_cnt_block,shop_category_cnt_block
1534813,15481,21,27,1,63,4,2015,18.0,2053.0,1316.0,68.0
1920333,19672,15,27,0,37,4,2015,9.0,1364.0,4670.0,118.0
293523,3235,53,27,0,31,4,2015,0.0,1322.0,2506.0,0.0
1016222,10645,38,26,0,67,3,2015,1.0,1543.0,674.0,11.0
449359,4548,55,33,0,75,10,2015,5.0,1972.0,929.0,0.0
921969,9565,10,33,0,40,10,2015,5.0,428.0,7680.0,69.0
1608755,16083,24,29,0,65,6,2015,12.0,882.0,1941.0,5.0
539977,5469,42,31,0,58,8,2015,6.0,3832.0,284.0,27.0
779113,8077,3,27,0,40,4,2015,3.0,740.0,10683.0,62.0
2069470,21283,21,24,0,61,1,2015,1.0,1923.0,744.0,18.0


In [ ]:
np.average(training[(training['item_id'] == 491) & (training['shop_id'] == 2)]['item_cnt_block'].values, weights=[1/55,2/55,3/55,4/55,5/55,6/55,7/55,8/55,9/55,10/55])

In [22]:
weights=[1/55,2/55,3/55,4/55,5/55,6/55,7/55,8/55,9/55,10/55]
item_cnt_block_avg = training.groupby(['item_id', 'shop_id'])['item_cnt_block'].apply(np.average, weights=weights).reset_index().rename(columns={"item_cnt_block":"item_cnt_block_avg"})
shop_cnt_block_avg = training.groupby(['item_id', 'shop_id'])['shop_cnt_block'].apply(np.average, weights=weights).reset_index().rename(columns={"shop_cnt_block":"shop_cnt_block_avg"})
category_cnt_block_avg = training.groupby(['item_id', 'shop_id'])['category_cnt_block'].apply(np.average, weights=weights).reset_index().rename(columns={"category_cnt_block":"category_cnt_block_avg"})
shop_cat_cnt_block_avg = training.groupby(['item_id', 'shop_id'])['shop_category_cnt_block'].apply(np.average, weights=weights)\
                    #.reset_index().rename(columns={"shop_category_cnt_block":"shop_category_cnt_block_avg"})

In [23]:
item_cnt_block_avg.head()
consolidated = item_cnt_block_avg
cols= ["item_id", "shop_id"]
consolidated = consolidated.merge(shop_cnt_block_avg,on=cols,how='left')
consolidated = consolidated.merge(category_cnt_block_avg,on=cols,how='left')
consolidated = consolidated.merge(shop_cat_cnt_block_avg,on=cols,how='left')

In [28]:
consolidated = consolidated.merge(training[training['date_block_num'] == 33][cols+["shop_item_cnt_block"]],on=cols,how='left')

In [29]:
#consolidated.drop(columns=["shop_item_cnt_block"],inplace=True)
consolidated.head()

,item_id,shop_id,item_cnt_block_avg,shop_cnt_block_avg,category_cnt_block_avg,shop_category_cnt_block_avg,shop_item_cnt_block
0,30,2,4.272727,822.927273,8982.781818,66.672727,0
1,30,2,4.272727,822.927273,8982.781818,66.672727,0
2,30,2,4.272727,822.927273,8982.781818,66.672727,0
3,30,2,4.272727,822.927273,8982.781818,66.672727,0
4,30,2,4.272727,822.927273,8982.781818,66.672727,0


In [30]:
training.columns.values

array(['item_id', 'shop_id', 'date_block_num', 'shop_item_cnt_block',
       'item_category_id', 'month', 'year', 'item_cnt_block',
       'shop_cnt_block', 'category_cnt_block', 'shop_category_cnt_block'],
      dtype=object)

In [ ]:
all_features = [
    
  'item_cnt_block',
       'shop_cnt_block', 'category_cnt_block', 'shop_category_cnt_block',
       'item_cnt_block_mean', 'item_cnt_block_min', 'item_cnt_block_max',
       'item_cnt_block_std', 'item_cnt_block_med', 'shop_cnt_block_mean',
       'shop_cnt_block_min', 'shop_cnt_block_max', 'shop_cnt_block_std',
       'shop_cnt_block_med', 'category_cnt_block_mean',
       'category_cnt_block_min', 'category_cnt_block_max',
       'category_cnt_block_std', 'category_cnt_block_med',
       'shop_category_cnt_block_mean', 'shop_category_cnt_block_min',
       'shop_category_cnt_block_max', 'shop_category_cnt_block_std',
       'shop_category_cnt_block_med', 'shop_item_cnt_block_mean',
       'shop_item_cnt_block_min', 'shop_item_cnt_block_max',
       'shop_item_cnt_block_std', 'shop_item_cnt_block_med',
       'item_id_mean_encoding', 'shop_id_mean_encoding',
       'item_category_id_mean_encoding'
]



features =  [
    
    'item_cnt_block',
       'shop_cnt_block', 'category_cnt_block', 'shop_category_cnt_block',
       'item_cnt_block_mean', 'item_cnt_block_min', 'item_cnt_block_max',
       'item_cnt_block_std', 'item_cnt_block_med', 'shop_cnt_block_mean',
       'shop_cnt_block_min', 'shop_cnt_block_max', 'shop_cnt_block_std',
       'shop_cnt_block_med', 'category_cnt_block_mean',
       'category_cnt_block_min', 'category_cnt_block_max',
       'category_cnt_block_std', 'category_cnt_block_med',
       'shop_category_cnt_block_mean', 'shop_category_cnt_block_min',
       'shop_category_cnt_block_max',
       'item_id_mean_encoding', 'shop_id_mean_encoding',
       'item_category_id_mean_encoding'
    
]

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler 


training[all_features] = MinMaxScaler().fit_transform(training[all_features])



In [ ]:
training[all_features] = training[all_features].apply(pd.to_numeric, downcast='float')

In [ ]:
training[features].sample(10)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3).fit(training[features])
print(pca.explained_variance_ratio_)

In [ ]:
training_pca = pca.transform(training[features])

In [ ]:
training['pca1'] = np.array(training_pca).T[0]
training['pca2'] = np.array(training_pca).T[1]
training['pca3'] = np.array(training_pca).T[2]

In [ ]:
x_train = training[(training['date_block_num'] < 33)]
y_train = x_train['shop_item_cnt_block']
x_train = x_train[['date_block_num', 'pca1', 'pca2', 'pca3']]




x_val = training[training['date_block_num'] == 33]
y_val = x_val['item_cnt_block']
x_val = x_val[['date_block_num', 'pca1', 'pca2', 'pca3']]

In [ ]:
len(x_train)

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression,BayesianRidge, HuberRegressor


lr_model =  LinearRegression()
lr_model.fit(x_train, y_train)

from sklearn.metrics import mean_squared_error
from math import sqrt

lr_val_preds = lr_model.predict(x_val)
lr_val_preds.clip(0,20,out=lr_val_preds)
rms = sqrt(mean_squared_error(y_val, lr_val_preds))
print("rmse: ", rms)

In [ ]:
lr_model.coef_

In [ ]:
test.head()

In [ ]:
training[(training['shop_id'] == 5) & ((training['item_id'] == 5037))][['date_block_num', 'pca1', 'pca2', 'pca3']]

In [ ]:
training_test = training[(training['shop_id'].isin(test['shop_id'].unique()))\
                         & (training['item_id'].isin(test['item_id'].unique()))]

In [ ]:
a = list(range(0, 235620, 21420))
b = list(range(21420, 257040, 21420))
intervals = list(zip(a,b))[:-1]
intervals

In [ ]:
import multiprocessing as mp

import importlib
import build_test_lr
importlib.reload(build_test_lr)


from build_test_lr import build_test_lr_f

features = ['pca1', 'pca2', 'pca3']

if __name__ ==  '__main__':
    pool = mp.Pool(mp.cpu_count())
    res = [pool.apply_async(build_test_lr_f,args=[interval, test, training_test, features]) for interval in intervals]
    pool.close()
    pool.join()

In [ ]:
test_data = []

for interval in intervals:
    for re in res:
        if interval in re.get():
            for sample in re.get()[interval]:
                test_data.append(sample)

In [ ]:
for i,component in enumerate(pca.explained_variance_ratio_):
    name = 'pca%d' % (i)
    test[name] = np.array(test_data).T[i]
    
test['date_block_num'] =  34

In [ ]:
preds = lr_model.predict(test[['date_block_num','pca1', 'pca2', 'pca3']])
preds.clip(0,20,out=preds)

In [ ]:

print(np.mean(preds))
print(np.max(preds))

In [ ]:
submission = test.loc[:,['ID']]
submission['item_cnt_month'] = preds

submission.to_csv('submission.csv', index=False)

In [ ]:
bestpreds = pd.read_csv('submissionbest.csv')['item_cnt_month']
print(np.mean(bestpreds))
print(np.max(bestpreds))

In [ ]:
lr_preds = pd.read_csv('lr110.csv')['item_cnt_month']
lg_preds = pd.read_csv('lg110.csv')['item_cnt_month']
#cb_preds = pd.read_csv('cb102.csv')['item_cnt_month']


#preds = np.mean(np.array([lr_preds, lg_preds]),axis=0)

preds = (lg_preds * 0.50) + (lr_preds * 0.50)